In [ ]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
from dash import html
#import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
launch_sites_array = spacex_df['Launch Site'].unique()
size = len(launch_sites_array)
size = size + 1
launch_sites_array.resize(size)
launch_sites_array[size-1] = 'All Sites'
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                      #{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': i, 'value': i} for i in launch_sites_array
                                                     ],
                                             value='ALL',
                                             placeholder="Select a Launch Site",
                                             searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000,
						        marks={0: '0',2500: '2500',5000: '5000',7500: '7500',10000: '10000'},
                				value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
        Output(component_id='success-pie-chart', component_property='figure'),
        Input(component_id='site-dropdown', component_property='value')
)

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'All Sites':
       fig = px.pie(
       data_frame = filtered_df,
       values='class', 
       names='Launch Site', 
       title='Launch Success rate')
       return(fig)	
    else:
       filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site] 
       fig = px.pie(
       data_frame = filtered_df,
       names='class', 
       title='Launch Success rate')
       return(fig)	

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
	[Input(component_id='site-dropdown', component_property='value'),
	 Input(component_id='payload-slider', component_property='value')]
)

def get_scatter_chart(entered_site,minmax):
    filtered_df = spacex_df
    fdf_bool = filtered_df['Payload Mass (kg)'].isin(range(minmax[0],minmax[1]))
    fdf2 = filtered_df[fdf_bool]
    print(fdf2.head())
    #fdf2 = filtered_df[(filtered_df.Payload Mass (kg) > minmax[0]) & (filtered_df.Payload Mass (kg) < minmax[1])]
    if entered_site == 'All Sites':
       fig = px.scatter(
       data_frame = fdf2,
       x ='Payload Mass (kg)', 
       y ='class', 
       title = 'Correlation between Payload and Success for all sites',
       color='Booster Version Category')
       return(fig)	
    else:
       filtered_df = spacex_df[spacex_df["Launch Site"] == entered_site] 
       fdf_bool = filtered_df['Payload Mass (kg)'].isin(range(minmax[0],minmax[1]))
       fdf2 = filtered_df[fdf_bool]
       #fdf2 = df[out]
       #fdf2 = filtered_df[(filtered_df.Payload Mass (kg) > minmax[0]) & (filtered_df.Payload Mass (kg) < minmax[1])]
       fig = px.scatter(
       data_frame = fdf2,
       x ='Payload Mass (kg)', 
       y ='class', 
       title = 'Correlation between Payload and Success for the selected site',
       color='Booster Version Category')
       return(fig)	

# Run the app
if __name__ == '__main__':
    app.run_server()
